In [ ]:
from datetime import datetime
import json
current = datetime.utcnow()
current

datetime.datetime(2023, 5, 24, 5, 54, 39, 604282)

In [ ]:
def format_iso(dt):
    return dt.strftime('%Y-%m-%dT%H:%M:%S')

In [ ]:
format_iso(current)

'2023-05-24T05:53:37'

In [ ]:
log_record = {'time': datetime.utcnow().isoformat(), 'message': 'testing'}
json.dumps(log_record)

'{"time": "2023-05-24T05:55:14.336457", "message": "testing"}'

Let's first write it without the decorator to make sure we have our code correct:

In [ ]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)

In [ ]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2023-05-24T05:55:14.336457", "message": "testing"}'

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return {
            'name': self.name,
            'age': self.age,
            'create_dt': self.create_dt.isoformat()
        }

In [ ]:
p = Person('John', 82)
print(p)
print(p.toJSON())

Person(name=John, age=82)
{'name': 'John', 'age': 82, 'create_dt': '2023-05-24T06:06:13.132241'}


In [ ]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    elif isinstance(arg, Person):
        return arg.toJSON()

In [ ]:
log_record = dict(time=datetime.utcnow(),
                  message='Created new person record',
                  person=p)

In [ ]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2023-05-24T06:06:54.530044", "message": "Created new person record", "person": {"name": "John", "age": 82, "create_dt": "2023-05-24T06:06:13.132241"}}'

In [ ]:
print(json.dumps(log_record, default=custom_json_formatter, indent=2))

{
  "time": "2023-05-24T06:06:54.530044",
  "message": "Created new person record",
  "person": {
    "name": "John",
    "age": 82,
    "create_dt": "2023-05-24T06:06:13.132241"
  }
}


In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)

In [ ]:
p = Person('Python', 27)

In [ ]:
log_record = dict(time=datetime.utcnow(),
                  message='Created new person record',
                  person=p)

In [ ]:
print(json.dumps(log_record, default=custom_json_formatter, indent=2))

{
  "time": "2023-05-24T06:08:57.118874",
  "message": "Created new person record",
  "person": {
    "name": "Python",
    "age": 27,
    "create_dt": "2023-05-24T06:08:18.633524"
  }
}


In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)

In [ ]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    else:
        try:
            return arg.toJSON()
        except AttributeError:
            try:
                return vars(arg)
            except TypeError:
                return str(arg)

In [ ]:
from functools import singledispatch

In [ ]:
from decimal import Decimal 
pt1 = Point(1,2)
p = Person('John',18)
pt2 = Point(Decimal('10.5'), Decimal(100.5))

log_record = dict(time=datetime.utcnow(),
                  message='Created new point',
                  point=pt1,
                  point_2 = pt2,
                  created_by=p)

In [ ]:
print(json.dumps(log_record, default=custom_json_formatter, indent=2))

{
  "time": "2023-05-24T06:19:13.058858",
  "message": "Created new point",
  "point": {
    "x": 1,
    "y": 2
  },
  "point_2": {
    "x": "10.5",
    "y": "100.5"
  },
  "created_by": {
    "name": "John",
    "age": 18,
    "create_dt": "2023-05-24T06:19:13.058567"
  }
}


In [ ]:
@singledispatch
def json_format(arg):
    print(arg)
    try:
        print('\ttrying to use toJSON...')
        return arg.toJSON()
    except AttributeError:
        print('\tfailed - trying to use vars...')
        try:
            return vars(arg)
        except TypeError:
            print('\tfailed - using string representation...')
            return str(arg)

In [ ]:
@json_format.register(datetime)
def _(arg):
    return arg.isoformat()

In [ ]:
@json_format.register(set)
def _(arg):
    return list(arg)

In [ ]:
log_record = dict(time=datetime.utcnow(),
                  message='Created new point',
                  point=pt1,
                  point_2 = pt2,
                  created_by=p)

In [ ]:
print(json.dumps(log_record, default=json_format, indent=2))

Point(x=1, y=2)
	trying to use toJSON...
	failed - trying to use vars...
Point(x=10.5, y=100.5)
	trying to use toJSON...
	failed - trying to use vars...
10.5
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
100.5
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
Person(name=John, age=18)
	trying to use toJSON...
{
  "time": "2023-05-24T06:19:13.058858",
  "message": "Created new point",
  "point": {
    "x": 1,
    "y": 2
  },
  "point_2": {
    "x": "10.5",
    "y": "100.5"
  },
  "created_by": {
    "name": "John",
    "age": 18,
    "create_dt": "2023-05-24T06:19:13.058567"
  }
}


#Let's change our Person class to emit some custom JSON instead of just using vars:

In [ ]:
from decimal import Decimal
from fractions import Fraction

json.dumps(dict(a=1+1j, 
                b=Decimal('0.5'), 
                c=Fraction(1, 3),
                p=Person('Python', 27),
                pt=Point(0,0),
                time=datetime.utcnow()
               ), 
           default=json_format)

(1+1j)
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
0.5
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
1/3
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
Person(name=Python, age=27)
	trying to use toJSON...
Point(x=0, y=0)
	trying to use toJSON...
	failed - trying to use vars...


'{"a": "(1+1j)", "b": "0.5", "c": "1/3", "p": {"name": "Python", "age": 27, "create_dt": "2023-05-24T06:36:54.868193"}, "pt": {"x": 0, "y": 0}, "time": "2023-05-24T06:36:54.868200"}'

In [ ]:
@json_format.register(Decimal)
def _(arg):
    return f'Decimal({str(arg)})'

In [ ]:
json.dumps(dict(a=1+1j, 
                b=Decimal(0.5), 
                c=Fraction(1, 3),
                p=Person('Python', 27),
                pt = Point(0,0),
                time = datetime.utcnow()
               ), 
           default=json_format)

(1+1j)
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
1/3
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
Person(name=Python, age=27)
	trying to use toJSON...
Point(x=0, y=0)
	trying to use toJSON...
	failed - trying to use vars...


'{"a": "(1+1j)", "b": "Decimal(0.5)", "c": "1/3", "p": {"name": "Python", "age": 27, "create_dt": "2023-05-24T06:38:31.273685"}, "pt": {"x": 0, "y": 0}, "time": "2023-05-24T06:38:31.273690"}'

In [ ]:
print(json.dumps(dict(pt = Point(Person('Python', 27), 2+2j)),
          default=json_format, indent=2))

Point(x=Person(name=Python, age=27), y=(2+2j))
	trying to use toJSON...
	failed - trying to use vars...
Person(name=Python, age=27)
	trying to use toJSON...
(2+2j)
	trying to use toJSON...
	failed - trying to use vars...
	failed - using string representation...
{
  "pt": {
    "x": {
      "name": "Python",
      "age": 27,
      "create_dt": "2023-05-24T06:38:42.022302"
    },
    "y": "(2+2j)"
  }
}
